In [71]:
import pandas as pd
import numpy as np
import pickle
from itertools import compress

In [2]:
with open('data/csgo_df.pkl', 'rb') as f:
    df_raw = pickle.load(f)

In [3]:
player_count = pd.read_csv('data/csgo.csv')

In [4]:
df = df_raw.copy()

#### List of items:

In [5]:
items = df.item_name.unique()

#### Possible parentheticals:

In [6]:
parentheticals = set([item[item.rfind('('):] for item in items if item.find('(') >= 0])
parentheticals

{'(Battle Green)',
 '(Battle-Scarred)',
 '(Bazooka Pink)',
 '(Blood Red)',
 '(Blue)',
 '(Brick Red)',
 '(Cash Green)',
 '(Desert Amber)',
 '(Dust Brown)',
 '(Factory New)',
 '(Field-Tested)',
 '(Foil)',
 '(Foil)  | MLG Columbus 2016',
 '(Foil) | Atlanta 2017',
 '(Foil) | Boston 2018',
 '(Foil) | Cluj-Napoca 2015',
 '(Foil) | Cologne 2015',
 '(Foil) | Cologne 2016',
 '(Foil) | DreamHack 2014',
 '(Foil) | Katowice 2014',
 '(Foil) | Katowice 2015',
 '(Foil) | Krakow 2017',
 '(Foil) | MLG Columbus 2016',
 '(Frog Green)',
 '(Gold) | Boston 2018',
 '(Gold) | Krakow 2017',
 '(Holo)',
 '(Holo) | Atlanta 2017',
 '(Holo) | Boston 2018',
 '(Holo) | Cologne 2014',
 '(Holo) | Cologne 2016',
 '(Holo) | DreamHack 2014',
 '(Holo) | Katowice 2014',
 '(Holo) | Katowice 2015',
 '(Holo) | Krakow 2017',
 '(Holo) | MLG Columbus 2016',
 '(Holo-Foil)',
 '(Jungle Green)',
 '(Minimal Wear)',
 '(Monarch Blue)',
 '(Monster Purple)',
 '(Princess Pink)',
 '(Red)',
 '(SWAT Blue)',
 '(Shark White)',
 '(Tiger Orange)'

#### We have colors, quality, foil, events (after '|', and likely separate of parentheticals if foil is missing)

In [7]:
list(compress(items, ['(Foil)  | MLG Columbus 2016' in item for item in items]))

['Sticker | AdreN (Foil)  | MLG Columbus 2016']

In [8]:
'Sticker | AdreN (Foil) | MLG Columbus 2016'

'Sticker | AdreN (Foil) | MLG Columbus 2016'

In [9]:
df = df.replace(to_replace='Sticker | AdreN (Foil)  | MLG Columbus 2016', value='Sticker | AdreN (Foil) | MLG Columbus 2016')

#### Removed the extra space in that string

#### Looking at the items without parentheses

In [10]:
df['parenthetical'] = [item.find('(') >= 0 for item in df.item_name]

In [11]:
len(df[~df.parenthetical].item_name.unique())

1450

1450 unique items left to categorize

In [12]:
df.head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical
604327,730,1.471583e+09,Glock-18 | Weasel (Battle-Scarred),6.193,135,18 August 2016,True
604328,730,1.471669e+09,Glock-18 | Weasel (Battle-Scarred),5.103,166,18 August 2016,True
604329,730,1.471756e+09,Glock-18 | Weasel (Battle-Scarred),4.721,142,18 August 2016,True
604330,730,1.471842e+09,Glock-18 | Weasel (Battle-Scarred),4.302,98,18 August 2016,True
604331,730,1.471928e+09,Glock-18 | Weasel (Battle-Scarred),3.898,125,18 August 2016,True


In [13]:
gun_list = ['AK-47',
 'AUG',
 'AWP',
 'CZ75-Auto',
 'Desert Eagle',
 'Dual Berettas',
 'FAMAS',
 'Five-SeveN',
 'G3SG1',
 'Galil AR',
 'Glock-18',
 'M249',
 'M4A1-S',
 'M4A4',
 'MAC-10',
 'MAG-7',
 'MP7',
 'MP9',
 'Negev',
 'Nova',
 'P2000',
 'P250',
 'P90',
 'PP-Bizon',
 'R8 Revolver',
 'SCAR-20',
 'SG 553',
 'SSG 08',
 'Sawed-Off',
 'Tec-9',
 'UMP-45',
 'USP-S',
 'XM1014']

#### Add guns column

In [59]:
df['gun_bool'] = [any([x in y for x in gun_list]) for y in df.item_name]

In [62]:
df.gun.tail()

9486136    False
9486137    False
9486138    False
9486139    False
9486140    False
Name: gun, dtype: bool

In [61]:
sorted(set([x.split('|')[0][:-1] for x in df[df.gun].item_name.unique()]))

['AK-47',
 'AUG',
 'AWP',
 'CZ75-Auto',
 'Desert Eagle',
 'Dual Berettas',
 'FAMAS',
 'Five-SeveN',
 'G3SG1',
 'Galil AR',
 'Glock-18',
 'M249',
 'M4A1-S',
 'M4A4',
 'MAC-10',
 'MAG-7',
 'MP7',
 'MP9',
 'Negev',
 'Nova',
 'P2000',
 'P250',
 'P90',
 'PP-Bizon',
 'R8 Revolver',
 'SCAR-20',
 'SG 553',
 'SSG 08',
 'Sawed-Off',
 'Tec-9',
 'UMP-45',
 'USP-S',
 'XM1014']

#### Separate StatTraks and Souvenirs

In [52]:
df['StatTrak'] = [x.find('Stat') == 0 for x in df.item_name]

In [53]:
df.loc[df.StatTrak, 'item_name'] = [x.split('StatTrak™ ')[1] for x in df.loc[df.StatTrak, 'item_name']]

In [54]:
df[df.StatTrak].head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak
612956,730,1.471583e+09,P90 | Grim (Field-Tested),7.559,153,18 August 2016,True,True
612957,730,1.471669e+09,P90 | Grim (Field-Tested),5.642,179,18 August 2016,True,True
612958,730,1.471756e+09,P90 | Grim (Field-Tested),4.711,150,18 August 2016,True,True
612959,730,1.471842e+09,P90 | Grim (Field-Tested),4.308,140,18 August 2016,True,True
612960,730,1.471928e+09,P90 | Grim (Field-Tested),3.919,138,18 August 2016,True,True


#### Separate Souvenir

In [56]:
df['souvenir'] = [x.find('Souv') == 0 for x in df.item_name]

In [57]:
df.loc[df.souvenir, 'item_name'] = [x.split('Souvenir ')[1] for x in df.loc[df.souvenir, 'item_name']]

In [66]:
df[df.souvenir].head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak,souvenir,gun
799720,730,1.416722e+09,P250 | Nuclear Threat (Minimal Wear),234.660,1,14 August 2013,True,False,True,True
799721,730,1.419746e+09,P250 | Nuclear Threat (Minimal Wear),399.996,1,14 August 2013,True,False,True,True
799722,730,1.420178e+09,P250 | Nuclear Threat (Minimal Wear),399.996,1,14 August 2013,True,False,True,True
799723,730,1.421993e+09,P250 | Nuclear Threat (Minimal Wear),302.380,1,14 August 2013,True,False,True,True
799724,730,1.426396e+09,P250 | Nuclear Threat (Minimal Wear),139.219,2,14 August 2013,True,False,True,True


#### Separate Gun, Skin, quality

In [67]:
df.loc[799720].date

1416722400.0

In [77]:
df['gun_type'], df['skin'], df['quality'] = np.nan, np.nan, np.nan

In [78]:
df.head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak,souvenir,gun_bool,gun_type,skin,quality
604327,730,1.471583e+09,Glock-18 | Weasel (Battle-Scarred),6.193,135,18 August 2016,True,False,False,True,NaN,NaN,NaN
604328,730,1.471669e+09,Glock-18 | Weasel (Battle-Scarred),5.103,166,18 August 2016,True,False,False,True,NaN,NaN,NaN
604329,730,1.471756e+09,Glock-18 | Weasel (Battle-Scarred),4.721,142,18 August 2016,True,False,False,True,NaN,NaN,NaN
604330,730,1.471842e+09,Glock-18 | Weasel (Battle-Scarred),4.302,98,18 August 2016,True,False,False,True,NaN,NaN,NaN
604331,730,1.471928e+09,Glock-18 | Weasel (Battle-Scarred),3.898,125,18 August 2016,True,False,False,True,NaN,NaN,NaN


In [79]:
df.loc[df.gun_bool, 'gun_type'] = [x.split(' |')[0] for x in df.item_name[df.gun_bool]]

In [84]:
df.loc[df.gun_bool, 'skin'] = [x.split('| ')[1][:x.split('| ')[1].rfind('(')] for x in df.item_name[df.gun_bool]]

In [82]:
df.loc[df.gun_bool, 'quality'] = [x.split('(')[-1][:-1] for x in df.item_name[df.gun_bool]]

In [93]:
df.head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak,souvenir,gun_bool,gun_type,skin,quality
604327,730,1.471583e+09,Glock-18 | Weasel (Battle-Scarred),6.193,135,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred
604328,730,1.471669e+09,Glock-18 | Weasel (Battle-Scarred),5.103,166,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred
604329,730,1.471756e+09,Glock-18 | Weasel (Battle-Scarred),4.721,142,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred
604330,730,1.471842e+09,Glock-18 | Weasel (Battle-Scarred),4.302,98,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred
604331,730,1.471928e+09,Glock-18 | Weasel (Battle-Scarred),3.898,125,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred


#### Keys

In [100]:
df['keys'] = [x.find('Key') > -1 for x in df.item_name]

#### Capsules

In [109]:
df['capsules'] = [(x.find('Capsule') > -1) and not y for x, y in zip(df.item_name, df['keys'])]

#### Stickers

In [114]:
df['stickers'] = [(x.find('Sticker') >= 0) and not y and not z for x, y, z in zip(df.item_name, df['keys'], df['capsules'])]

In [115]:
df.head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak,souvenir,gun_bool,gun_type,skin,quality,keys,capsules,stickers
604327,730,1.471583e+09,Glock-18 | Weasel (Battle-Scarred),6.193,135,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred,False,False,False
604328,730,1.471669e+09,Glock-18 | Weasel (Battle-Scarred),5.103,166,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred,False,False,False
604329,730,1.471756e+09,Glock-18 | Weasel (Battle-Scarred),4.721,142,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred,False,False,False
604330,730,1.471842e+09,Glock-18 | Weasel (Battle-Scarred),4.302,98,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred,False,False,False
604331,730,1.471928e+09,Glock-18 | Weasel (Battle-Scarred),3.898,125,18 August 2016,True,False,False,True,Glock-18,Weasel,Battle-Scarred,False,False,False


#### Tournaments

In [131]:
tournaments = ['2018 ELEAGUE Boston',
'2017 PGL Krakow',
'2015 ESL One Cologne', 
'2017 ELEAGUE Atlanta',
'2016 MLG Columbus',
'2016 ESL One Cologne',
'2015 DreamHack Cluj-Napoca', 
'2015 ESL One Katowice',
'2014 ESL One Cologne',
'2014 EMS One Katowice',
'2014 DreamHack Winter',
'2013 DreamHack Winter']

In [132]:
tournaments = [x[5:] + ' ' + x[0:4] for x in tournaments]

In [133]:
tournaments

['ELEAGUE Boston 2018',
 'PGL Krakow 2017',
 'ESL One Cologne 2015',
 'ELEAGUE Atlanta 2017',
 'MLG Columbus 2016',
 'ESL One Cologne 2016',
 'DreamHack Cluj-Napoca 2015',
 'ESL One Katowice 2015',
 'ESL One Cologne 2014',
 'EMS One Katowice 2014',
 'DreamHack Winter 2014',
 'DreamHack Winter 2013']

In [134]:
df[[any([x in y for x in tournaments]) for y in df.item_name]].head()

,app,date,item_name,median_sell_price,quantity,release_date,parenthetical,StatTrak,souvenir,gun_bool,gun_type,skin,quality,keys,capsules,stickers
778352,730,1.458277e+09,Sticker | hooch (Foil) | MLG Columbus 2016,1.478,40,NaN,True,False,False,False,NaN,NaN,NaN,False,False,True
778353,730,1.458364e+09,Sticker | hooch (Foil) | MLG Columbus 2016,1.550,13,NaN,True,False,False,False,NaN,NaN,NaN,False,False,True
778354,730,1.458450e+09,Sticker | hooch (Foil) | MLG Columbus 2016,1.243,13,NaN,True,False,False,False,NaN,NaN,NaN,False,False,True
778355,730,1.458536e+09,Sticker | hooch (Foil) | MLG Columbus 2016,1.356,17,NaN,True,False,False,False,NaN,NaN,NaN,False,False,True
778356,730,1.458623e+09,Sticker | hooch (Foil) | MLG Columbus 2016,1.458,5,NaN,True,False,False,False,NaN,NaN,NaN,False,False,True


#### Put item_name back together

In [ ]:
df.loc[:,'item_name'] = df_raw['item_name']
df = df.reset_index()